<a href="https://colab.research.google.com/github/sorengod/MasterThesis/blob/main/Prepare_ABT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
import pandas as pd

In [170]:
Spot_Prices = pd.read_parquet("Spot_Prices.parq")
Import_Export = pd.read_parquet("ExportCapacity.parq")
Import_Capacity = pd.read_parquet("ImportCapacity.parq")
MeanWindPower1h_Anholdt = pd.read_pickle("MeanWindPower1h_Anholdt.pkl")
MeanWindPower1h_Blåvand = pd.read_pickle("MeanWindPower1h_Blåvand.pkl")
MeanWindPower1h_Gedser = pd.read_pickle("MeanWindPower1h_Gedser.pkl")
MeanWindPower1h_Køge = pd.read_pickle("MeanWindPower1h_Køge.pkl")
Production_Consumption_DK1 = pd.read_pickle("Production&Consumption_DK1.pkl")
Production_Consumption_DK2 = pd.read_pickle("Production&Consumption_DK2.pkl")
ScheduledExchangeDayAhead = pd.read_parquet("ScheduledExchangeDayAhead.parq")
 

# Prepare Import_Export


In [171]:
Import_Export = Import_Export.drop(['HourUTC', 'PhysicalExchangeNonvalidated', 'PhysicalExchangeSettlement', 
                                    'CongestionIncomeDKK', 'HomePriceDKK', 'ConnectedPriceDKK', 
                                    'CongestionIncomeEUR', 'HomePriceEUR', 'ConnectedPriceEUR'
                                    , 'ImportCapacity', 'ExportCapacity'], axis=1)


In [172]:
def prepare_Import_Export(Import_Export_df:pd.DataFrame, GridFlow:str): 
  converted_GridFlowStr = GridFlow.replace("-->", "_")
  df = Import_Export_df[Import_Export_df["GridFlow"].isin([f"{GridFlow}"])]
  df = df.rename(columns={"ScheduledExchangeDayAhead":f"ScheduledExchangeDayAhead_{converted_GridFlowStr}"})                                   
  df = df.drop(["GridFlow", "ScheduledExchangeIntraday"], axis=1)
  return df

In [173]:
Import_Export_DK1_DE = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK1-->DE")
Import_Export_DK1_DK2 = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK1-->DK2")
Import_Export_DK1_NO2 = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK1-->NO2")
Import_Export_DK1_SE3 = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK1-->SE3")
Import_Export_DK2_DE = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK2-->DE")
Import_Export_DK2_DK1 = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK2-->DK1")
Import_Export_DK2_SE4 = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK2-->SE4")
Import_Export_DK1_NL = prepare_Import_Export(Import_Export_df=Import_Export, GridFlow="DK1-->NL")

In [174]:
Import_Export_prepard = Import_Export_DK1_DK2.merge(Import_Export_DK1_DE, left_on='HourDK', right_on='HourDK', how='left')
Import_Export_prepard = Import_Export_prepard.merge(Import_Export_DK1_NO2, left_on='HourDK', right_on='HourDK', how='left')
Import_Export_prepard = Import_Export_prepard.merge(Import_Export_DK1_SE3, left_on='HourDK', right_on='HourDK', how='left')
Import_Export_prepard = Import_Export_prepard.merge(Import_Export_DK2_DE, left_on='HourDK', right_on='HourDK', how='left')
Import_Export_prepard = Import_Export_prepard.merge(Import_Export_DK2_SE4, left_on='HourDK', right_on='HourDK', how='left')
Import_Export_prepard = Import_Export_prepard.merge(Import_Export_DK1_NL, left_on='HourDK', right_on='HourDK', how='left')

Import_Export_prepared = Import_Export_prepard.round(2)

In [ ]:
Import_Export_prepared

#Prepare Wind Data


In [176]:
MeanWindPower1h_Anholdt

def prepare_wind_data(MeanWindPower_df:pd.DataFrame, placement:str):
  location = str(placement)
  df = MeanWindPower_df
  df = df.drop(["created", "parameterId", "stationId"], axis=1)
  df = df.rename(columns={"value" : f"value_{location}"})
  return df

In [177]:
Wind_Anholdt = prepare_wind_data(MeanWindPower1h_Anholdt, "Anholdt")
Wind_Blåvand = prepare_wind_data(MeanWindPower1h_Blåvand, "Blåvand")
Wind_Gedser = prepare_wind_data(MeanWindPower1h_Gedser, "Gedser")
Wind_Køge = prepare_wind_data(MeanWindPower1h_Køge, "Køge")

In [178]:
wind_prepared = Wind_Gedser.merge(Wind_Anholdt, left_on="observed", right_on="observed", how='left')
wind_prepared = wind_prepared.merge(Wind_Blåvand, left_on="observed", right_on="observed", how='left')
wind_prepared = wind_prepared.merge(Wind_Køge, left_on="observed", right_on="observed",how='left')

In [ ]:
wind_prepared

#Prepare Production And Consumption

In [ ]:
Production_Consumption_DK1.drop(columns=["HourUTC", "PriceArea", "ExchangeNO_MWh", "ExchangeSE_MWh", "ExchangeGE_MWh", "ExchangeNL_MWh", "ExchangeGreatBelt_MWh"], axis=1)

#Prepare ABT

In [189]:
abt = Spot_Prices.merge(Import_Export_prepared, left_on="HourDK", right_on="HourDK", how='left')
abt = abt.merge(Production_Consumption_DK1, left_on="HourDK", right_on="HourDK", how='left')

In [192]:
abt = abt.merge(wind_prepared, left_on="HourDK", right_on="observed", how='left')

In [196]:
abt.to_pickle("abt.pkl")

In [197]:
from google.colab import files
files.download('abt.pkl') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>